In [1]:
import pandas as pd
from langchain_community.document_loaders import TextLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import Chroma
import pandas as pd
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")



c:\Users\mloyolaf\OneDrive - NTT DATA EMEAL\Escritorio\Ignacia\Workbench\asistente_demo_datapalooza\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
loader = PyPDFLoader("../Resources/GCP.pdf")
data = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 500,
        length_function = len,
        chunk_overlap = 250
)

documents = text_splitter.split_documents(data)

In [ ]:
def dataframe_documentation(documents:list):
    data = pd.DataFrame()
    page_content = []
    metadata = []
    for line in range(len(documents)):
        page_content.append(documents[line].page_content)
        metadata.append(documents[line].metadata)
    data["page_content"] = page_content
    data["metadata"] = metadata
    return data



In [3]:
documents_data = dataframe_documentation(documents)
documents_data.head(2)

,page_content,metadata
0,Visualizing\nVergadia816324_ffirs.indd 1 25-...,"{'source': '../Resources/GCP.pdf', 'page': 2}"
1,Vergadia816324_ffirs.indd 2 25-02-2022 15:...,"{'source': '../Resources/GCP.pdf', 'page': 3}"


In [ ]:
# Creando los nuevos embeddings 

def created_embeddings(data:object, embeddings=embedding_function):
    col_index = data.columns.get_loc('page_content')
    vectors = []
    for index in range(data.shape[0]):
        val = embeddings.embed_query(data.iloc[index, col_index])
        vectors.append(val)
    data["embeddings"] = vectors
    return data

In [4]:
data_embeddings = created_embeddings(documents_data, embedding_function)
data_embeddings

,page_content,metadata,embeddings
0,Visualizing\nVergadia816324_ffirs.indd 1 25-...,"{'source': '../Resources/GCP.pdf', 'page': 2}","[0.017832061275839806, 0.02583828940987587, -0..."
1,Vergadia816324_ffirs.indd 2 25-02-2022 15:...,"{'source': '../Resources/GCP.pdf', 'page': 3}","[-0.036654166877269745, 0.03949139639735222, -..."
2,Priyanka Vergadia\nIllustrated References for ...,"{'source': '../Resources/GCP.pdf', 'page': 4}","[-0.017715079709887505, 0.07079383730888367, -..."
3,"Copyright © 2022 by Google, LLC. All rights re...","{'source': '../Resources/GCP.pdf', 'page': 5}","[-0.0044191270135343075, -0.021689217537641525..."
4,ISBN: 978- 1- 119- 81637- 9 (ebk)\nISBN: 978- ...,"{'source': '../Resources/GCP.pdf', 'page': 5}","[-0.07486551254987717, 0.046314872801303864, -..."
...,...,...,...
1762,Security 247\nSecurity Monitoring and Operatio...,"{'source': '../Resources/GCP.pdf', 'page': 256}","[-0.02674219012260437, -0.05542406812310219, -..."
1763,• Security Command Center: Continuously monito...,"{'source': '../Resources/GCP.pdf', 'page': 256}","[-0.00578530877828598, -0.055202335119247437, ..."
1764,activities and accesses within your Google Clo...,"{'source': '../Resources/GCP.pdf', 'page': 256}","[-0.04275853559374809, -0.009766227565705776, ..."
1765,"native network threat detection from malware, ...","{'source': '../Resources/GCP.pdf', 'page': 256}","[-0.090774767100811, -0.05555233359336853, 0.0..."


In [5]:
#data_embeddings.to_pickle("../Resources/data_embeddings.pickle")